In [103]:
import pandas as pd
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor as gp
from sklearn.preprocessing import StandardScaler as ss
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process.kernels import WhiteKernel as WK
from sklearn.gaussian_process.kernels import ConstantKernel as C
from sklearn.gaussian_process.kernels import ExpSineSquared as ESS

In [2]:
x = pd.read_csv('features.csv', encoding="latin1")

In [3]:
print(set(x['article_group0_idx']))

set([u'HofKonditorei', u'Trockenprodukte', u'Erlebnis', u'Non-Food', u'Getr\xe4nke', u'Obst', u'HofChuchi', u'Frischprodukte', u'HofB\xe4ckerei', u'Gem\xfcse'])


In [4]:
x_juck_res = x[(x['article_group0_idx'] == 'HofChuchi') & (x['shop_idx'] == u'Juckerhof Seegräben') & (x['is_resto'] == True) & (~ x['temperature'].isna())]      

In [5]:
x1 = x_juck_res.fillna(0)
x1.columns

Index([u'article_group0_idx', u'month_idx', u'hour_idx', u'mday_idx',
       u'year_idx', u'is_resto_idx', u'shop_idx', u'temperature',
       u'precipitation', u'snowfraction', u'sealevelpressure',
       u'winddirection', u'windspeed', u'relativehumidity', u'sunshinetime',
       u'totalcloudcover', u'lowclouds', u'midclouds', u'highclouds',
       u'wday_x', u'month', u'hour', u'mday', u'year', u'month.1', u'hour.1',
       u'mday.1', u'year.1', u'is_resto', u'price', u'count',
       u'n_transactions'],
      dtype='object')

In [82]:


y = x1['price']
X = x1[['month', 'hour.1', 'mday', 'wday_x', 'temperature', 'precipitation', 'sunshinetime']]

one_hot = pd.get_dummies(X['wday_x'], prefix='day')
X = X.drop('wday_x', axis=1)
X = X.join(one_hot)

one_hot = pd.get_dummies(X['month'], prefix='month')
X = X.drop('month', axis=1)
X = X.join(one_hot)

one_hot = pd.get_dummies(X['hour.1'], prefix='hour')
X = X.drop('hour.1', axis=1)
X = X.join(one_hot)



In [92]:
np.random.seed(1234)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
normalizer = ss()
X_train = normalizer.fit_transform(X_train)
X_test = normalizer.transform(X_test)

In [105]:

# cross validation 

lss = np.logspace(0,1,10)
nls = np.logspace(-1,1,10)


RBFS = map(lambda x: RBF(length_scale=x, length_scale_bounds='fixed') + WK(noise_level=1, noise_level_bounds='fixed'), np.logspace(0,1,10))
WS = map(lambda x: RBF(length_scale=3.56, length_scale_bounds='fixed') + WK(noise_level=x, noise_level_bounds='fixed'), np.logspace(-1,1,10))
Materns = map(lambda x: Matern(length_scale=x, length_scale_bounds='fixed') + WK(noise_level=1, noise_level_bounds='fixed'), np.logspace(-1,2,30))



params = {'kernel': Materns}

model = gp(random_state=1, normalize_y=True)
cv = RandomizedSearchCV(model, param_distributions=params, n_iter=10, cv=10, verbose=50, n_jobs=4)

cv.fit(X_train, y_train)
cv.best_estimator_.score(X_test, y_test)

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] kernel=Matern(length_scale=100, nu=1.5) + WhiteKernel(noise_level=1) 
[CV] kernel=Matern(length_scale=100, nu=1.5) + WhiteKernel(noise_level=1) 
[CV] kernel=Matern(length_scale=100, nu=1.5) + WhiteKernel(noise_level=1) 
[CV] kernel=Matern(length_scale=100, nu=1.5) + WhiteKernel(noise_level=1) 
[CV]  kernel=Matern(length_scale=100, nu=1.5) + WhiteKernel(noise_level=1), score=0.298708472478, total=   3.5s
[CV] kernel=Matern(length_scale=100, nu=1.5) + WhiteKernel(noise_level=1) 
[CV]  kernel=Matern(length_scale=100, nu=1.5) + WhiteKernel(noise_level=1), score=0.249039258855, total=   3.5s
[CV] kernel=Matern(length_scale=100, nu=1.5) + WhiteKernel(noise_level=1) 
[CV]  kernel=Matern(length_scale=100, nu=1.5) + WhiteKernel(noise_level=1), score=0.331981491178, total=   3.6s
[CV] kernel=Matern(length_scale=100, nu=1.5) + WhiteKernel(noise_level=1) 
[CV]  kernel=Matern(length_scale=100, nu=1.5) + WhiteKernel(noise_level=1), 

[CV] kernel=Matern(length_scale=0.329, nu=1.5) + WhiteKernel(noise_level=1) 
[Parallel(n_jobs=4)]: Done  32 tasks      | elapsed:   40.2s
[CV]  kernel=Matern(length_scale=0.329, nu=1.5) + WhiteKernel(noise_level=1), score=0.00123688787097, total=   3.8s
[CV] kernel=Matern(length_scale=0.329, nu=1.5) + WhiteKernel(noise_level=1) 
[CV]  kernel=Matern(length_scale=0.329, nu=1.5) + WhiteKernel(noise_level=1), score=0.00442319042802, total=   4.4s
[CV]  kernel=Matern(length_scale=0.329, nu=1.5) + WhiteKernel(noise_level=1), score=0.00320164662788, total=   3.7s
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:   42.5s
[Parallel(n_jobs=4)]: Done  34 tasks      | elapsed:   42.6s
[Parallel(n_jobs=4)]: Done  35 tasks      | elapsed:   42.7s
[CV] kernel=Matern(length_scale=0.329, nu=1.5) + WhiteKernel(noise_level=1) 
[CV] kernel=Matern(length_scale=0.329, nu=1.5) + WhiteKernel(noise_level=1) 
[CV]  kernel=Matern(length_scale=0.329, nu=1.5) + WhiteKernel(noise_level=1), score=0.00338380123495

[CV]  kernel=Matern(length_scale=0.127, nu=1.5) + WhiteKernel(noise_level=1), score=-0.00404635948199, total=   3.7s
[CV] kernel=Matern(length_scale=0.127, nu=1.5) + WhiteKernel(noise_level=1) 
[Parallel(n_jobs=4)]: Done  65 tasks      | elapsed:  1.4min
[CV]  kernel=Matern(length_scale=0.127, nu=1.5) + WhiteKernel(noise_level=1), score=-0.00146826428718, total=   4.0s
[CV] kernel=Matern(length_scale=0.127, nu=1.5) + WhiteKernel(noise_level=1) 
[Parallel(n_jobs=4)]: Done  66 tasks      | elapsed:  1.4min
[CV]  kernel=Matern(length_scale=0.127, nu=1.5) + WhiteKernel(noise_level=1), score=-0.00523944141668, total=   3.4s
[CV] kernel=Matern(length_scale=11.7, nu=1.5) + WhiteKernel(noise_level=1) 
[Parallel(n_jobs=4)]: Done  67 tasks      | elapsed:  1.4min
[CV]  kernel=Matern(length_scale=0.127, nu=1.5) + WhiteKernel(noise_level=1), score=-0.0115910500122, total=   4.3s
[CV] kernel=Matern(length_scale=11.7, nu=1.5) + WhiteKernel(noise_level=1) 
[Parallel(n_jobs=4)]: Done  68 tasks      | 

[CV]  kernel=Matern(length_scale=0.853, nu=1.5) + WhiteKernel(noise_level=1), score=0.127285297805, total=   2.6s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:  2.1min finished


0.59003526732412059

In [101]:
seeds = np.arange(10)
scores = []
for seed in seeds: 
    np.random.seed(seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    normalizer = ss()
    X_train = normalizer.fit_transform(X_train)
    X_test = normalizer.transform(X_test)
    kernel = RBF(length_scale=3.56, length_scale_bounds='fixed') + WK(noise_level=0.546, noise_level_bounds='fixed')

    model = gp(kernel=kernel).fit(X_train, y_train)
    model.fit(X_train, y_train)
    scores.append(model.score(X_test, y_test))

In [102]:
print(np.mean(scores), np.std(scores))

(0.62449261429196146, 0.026598768825192468)


In [79]:
n_features = X_train.shape[1]
kernel = C(1.0, constant_value_bounds="fixed") * RBF(1.0, length_scale_bounds="fixed") + ESS()
model = gp(kernel=kernel, random_state=1)
#model = gp(kernel=RBF(np.ones(n_features)))
model.fit(X_train, y_train)
model.score(X_test, y_test)

LinAlgError: ("The kernel, 1**2 * RBF(length_scale=1) + ExpSineSquared(length_scale=1, periodicity=1), is not returning a positive definite matrix. Try gradually increasing the 'alpha' parameter of your GaussianProcessRegressor estimator.", '12-th leading minor of the array is not positive definite')

In [124]:
from sklearn.gaussian_process.kernels import CompoundKernel
from sklearn.gaussian_process.kernels import WhiteKernel
from sklearn.gaussian_process.kernels import ExpSineSquared
from sklearn.gaussian_process.kernels import RBF

rbf = RBF()
ess = ExpSineSquared()
wk = WhiteKernel()
ck = CompoundKernel({'k1': rbf, 'k2':ess, 'k3': wk})

model2 = gp(kernel=ck)
model2.fit(X_train, y_train)
model2.score(X_test, y_test)

AttributeError: 'str' object has no attribute 'theta'

In [74]:
cv.best_params_

{'kernel': RBF(length_scale=3.56) + WhiteKernel(noise_level=0.546)}

In [66]:
cv.cv_results_

/usr/local/lib/python2.7/dist-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/deprecation.py:1

{'mean_fit_time': array([ 2.52372966,  2.59393964,  2.68270388,  2.79936523,  2.70368147,
         2.63490114,  2.68024678,  2.68676801,  2.73868828,  2.37268596]),
 'mean_score_time': array([ 0.17102318,  0.15184665,  0.1672616 ,  0.16661377,  0.18991356,
         0.22867179,  0.19662938,  0.26125679,  0.18904738,  0.14975004]),
 'mean_test_score': array([-0.00202517, -0.00202517, -0.00202517, -0.00202517, -0.00202517,
        -0.00202492,  0.0159964 ,  0.25467955,  0.60453345,  0.49568194]),
 'mean_train_score': array([ 0.75      ,  0.75      ,  0.75      ,  0.75      ,  0.75      ,
         0.75000007,  0.75391046,  0.78864647,  0.77634207,  0.53941404]),
 'param_kernel': masked_array(data = [RBF(length_scale=0.001) + WhiteKernel(noise_level=1)
  RBF(length_scale=0.00278) + WhiteKernel(noise_level=1)
  RBF(length_scale=0.00774) + WhiteKernel(noise_level=1)
  RBF(length_scale=0.0215) + WhiteKernel(noise_level=1)
  RBF(length_scale=0.0599) + WhiteKernel(noise_level=1)
  RBF(length_sca

[RBF(length_scale=0.001),
 RBF(length_scale=0.00278),
 RBF(length_scale=0.00774),
 RBF(length_scale=0.0215),
 RBF(length_scale=0.0599),
 RBF(length_scale=0.167),
 RBF(length_scale=0.464),
 RBF(length_scale=1.29),
 RBF(length_scale=3.59),
 RBF(length_scale=10)]

(3227, 33)

[0 1 2 3 4 5 6 7 8 9]
